In [1]:
from lightrag import LightRAG, QueryParam
from lightrag.utils import EmbeddingFunc
from sentence_transformers import SentenceTransformer
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os
import logging
import json
logging.basicConfig(level=logging.ERROR)

load_dotenv()
gemini_key = os.getenv("GOOGLE_API_KEY")
async def llm_model_func(prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs) -> str:
    client = genai.Client(api_key=gemini_key)

    if history_messages is None:
        history_messages = []
    combined_prompt = ""
    if system_prompt:
        combined_prompt += f"{system_prompt}\n"
    for msg in history_messages:
        combined_prompt += f"{msg['role']}: {msg['content']}\n"
        
    combined_prompt += f"user: {prompt}"
    
    response = client.models.generate_content(
        model="gemini-1.5-pro",
        contents=[combined_prompt],
        config=types.GenerateContentConfig(max_output_tokens=1000, temperature=0.1),
    )
    return response.text

async def embedding_func(texts):
    model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
    embeddings = model.encode(texts, convert_to_numpy=True)
    return embeddings

rag: LightRAG = None

async def init_rag():
    global rag
    rag = LightRAG(
        working_dir="./data_indexing",
        llm_model_func=llm_model_func,
        embedding_func=EmbeddingFunc(
            embedding_dim=768,
            max_token_size=8192,
            func=embedding_func,
        ),
        auto_manage_storages_states=False,  
    )
    await rag.initialize_storages()

async def process_query(user_query: str) -> str:
    result = await rag.aquery(
        query=user_query,
        param=QueryParam(mode="mix", top_k=10, response_type="Bullet Points", stream=True),
    )
    return result
file_path = r".\data_indexing\kv_store_llm_response_cache.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)
if "mix" in data:
    data["mix"] = {}
with open(file_path, "w", encoding="utf-8") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)
    
await init_rag()

d:\hehe\custom_lib\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: Process 3028 Shared-Data created for Single Process
INFO: Process 3028 initialized updated flags for namespace: [full_docs]
INFO: Process 3028 ready to initialize storage namespace: [full_docs]
INFO: Process 3028 initialized updated flags for namespace: [text_chunks]
INFO: Process 3028 ready to initialize storage namespace: [text_chunks]
INFO: Process 3028 initialized updated flags for namespace: [entities]
INFO: Process 3028 initialized updated flags for namespace: [relationships]
INFO: Process 3028 initialized updated flags for namespace: [chunks]
INFO: Process 3028 initialized updated flags for namespace: [chunk_entity_relation]
INFO: Process 3028 initialized updated flags for namespace: [llm_response_cache]
INFO: Process 3028 ready to ini

In [2]:
user_query = "So sánh đại học báo chí và tuyên truyền và đại học quốc gia Hà Nội?"
result = await process_query(user_query)
print(result)

* **Xếp hạng:**
    * Đại học Quốc gia Hà Nội (VNU): Xếp hạng 851-900 thế giới (QS World University Rankings 2025), hạng 325 thế giới về phát triển bền vững, hạng 202 thế giới về kết quả tuyển dụng.  Lĩnh vực Khoa học Xã hội xếp hạng 501-600 thế giới, riêng ngành Xã hội học xếp hạng 301-375 thế giới.
    * Học viện Báo chí và Tuyên truyền (AJC): Xếp hạng trong top 50 trường đại học hàng đầu tại Việt Nam (thường ở vị trí 40-45).

* **Ngành học nổi bật:**
    * VNU: Công nghệ thông tin, Toán học, Kinh tế.
    * AJC: Báo chí (đặc biệt là Báo truyền hình), Quan hệ công chúng, Truyền thông đa phương tiện.

* **Điểm chuẩn năm 2024 (một số ngành):**
    * AJC:  Dao động từ 24,68 (Xây dựng Đảng và chính quyền nhà nước) đến 38,12 (Lịch sử). Các ngành báo chí nhóm ngành báo chí dao động từ 34,98 đến 37,25. Các ngành truyền thông dao động từ 28,05 đến 37,7.
    * VNU: Không có thông tin điểm chuẩn cụ thể từ nguồn dữ liệu.

* **Thông tin khác:**
    * VNU: Có thế mạnh về khoa học tự nhiên, công ng